In [ ]:
%load_ext autoreload
%autoreload 2
import toolkit as ftk

In [46]:
import pandas as pd
#.freqstr Q-DEC, M, Y-DEC, W-SUN, D (no B)
a = pd.period_range("2018-01-01", periods=5, freq="W")
a

PeriodIndex(['2018-01-01/2018-01-07', '2018-01-08/2018-01-14',
             '2018-01-15/2018-01-21', '2018-01-22/2018-01-28',
             '2018-01-29/2018-02-04'],
            dtype='period[W-SUN]')

In [48]:
import sys
sys.path

['/workspaces/financial-toolkit',
 '/workspaces/financial-toolkit/notebooks',
 '/home/codespace/.python/current/lib/python310.zip',
 '/home/codespace/.python/current/lib/python3.10',
 '/home/codespace/.python/current/lib/python3.10/lib-dynload',
 '',
 '/home/codespace/.local/lib/python3.10/site-packages',
 '/home/codespace/.python/current/lib/python3.10/site-packages']